Importing PyTorch and other important modules

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

Device Configuration

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Hyper Parameters

In [32]:
num_epochs = 10
batch_size = 5
learning_rate = 0.001

Transformer

In [33]:
# Dataset has PILImage of images of range [0, 1].
# We transform them to Tensors of normalized range [-1, 1].

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

Importing the CIFAR Dataset

In [34]:
train_dataset = torchvision.datasets.CIFAR10(root='./CIFAR', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./CIFAR', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [35]:
classes = ('Plane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'House', 'Ship', 'Truck')

Implementing the Convolutional Neural Network

In [36]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) # 3 input channels, 6 output channels, 5x5 kernel size
        self.pool = nn.MaxPool2d(2, 2) # 2x2 kernel size and stride of 2
        self.conv2 = nn.Conv2d(6, 16, 5) # 6 input channels, 16 output channels, 5x5 kernel size
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # Fully connected layer 1        
        self.fc2 = nn.Linear(120, 84) # Fully connected layer 2
        self.fc3 = nn.Linear(84, 10) # Fully connected layer 3, 10 because 10 different classes
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # Convolution -> ReLU -> Pooling
        x = self.pool(F.relu(self.conv2(x))) # Convolution -> ReLU -> Pooling
        x = x.view(-1, 16 * 5 * 5) # Flatten the tensor
        x = F.relu(self.fc1(x)) # Fully connected layer 1 -> ReLU
        x = F.relu(self.fc2(x)) # Fully connected layer 2 -> ReLU
        x = self.fc3(x) # Fully connected layer 3
        return x
    
model = ConvNet().to(device)

Loss and Optimizer

In [37]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Training Loop

In [38]:
total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Origin Shape : [4, 3, 32, 32] = 4, 3, 1024
        # Input Layer: 3 Input Channels, 6 Output Channels, 5 Kernel Size
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward Pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward Pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 1000 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_steps}], Loss: {loss.item():.4f}')
            
print("Finished Training")

Epoch [1/10], Step [1000/10000], Loss: 2.2788
Epoch [1/10], Step [2000/10000], Loss: 2.3020
Epoch [1/10], Step [3000/10000], Loss: 2.2476
Epoch [1/10], Step [4000/10000], Loss: 2.2988
Epoch [1/10], Step [5000/10000], Loss: 2.3162
Epoch [1/10], Step [6000/10000], Loss: 2.3136
Epoch [1/10], Step [7000/10000], Loss: 2.2830
Epoch [1/10], Step [8000/10000], Loss: 2.2759
Epoch [1/10], Step [9000/10000], Loss: 2.2565
Epoch [1/10], Step [10000/10000], Loss: 2.1875
Epoch [2/10], Step [1000/10000], Loss: 1.9657
Epoch [2/10], Step [2000/10000], Loss: 1.5606
Epoch [2/10], Step [3000/10000], Loss: 2.2168
Epoch [2/10], Step [4000/10000], Loss: 2.0860
Epoch [2/10], Step [5000/10000], Loss: 1.7632
Epoch [2/10], Step [6000/10000], Loss: 2.5107
Epoch [2/10], Step [7000/10000], Loss: 1.7151
Epoch [2/10], Step [8000/10000], Loss: 1.6992
Epoch [2/10], Step [9000/10000], Loss: 1.6865
Epoch [2/10], Step [10000/10000], Loss: 1.6567
Epoch [3/10], Step [1000/10000], Loss: 1.4870
Epoch [3/10], Step [2000/10000],

Evaluate the Model

In [39]:
model.eval()
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        # Max Returns (Value, Index)
        
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if(label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
    
    acc = 100 * n_correct / n_samples
    print(f'Accuracy on the test set: {acc} %')
    
    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy on the test set: 55.0 %
Accuracy of Plane: 56.5 %
Accuracy of Car: 55.6 %
Accuracy of Bird: 46.7 %
Accuracy of Cat: 35.3 %
Accuracy of Deer: 25.0 %
Accuracy of Dog: 39.4 %
Accuracy of Frog: 71.4 %
Accuracy of House: 67.0 %
Accuracy of Ship: 77.7 %
Accuracy of Truck: 75.4 %
